<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-11 10:14:33
-------------------
qualified stocks: 93
with latest results: 28
still star stocks: 18
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  2.51 L
Current:  1.32 C
-------------------
Today PnL: -63.86 K (-0.48%)
Current PnL: -23.18 L (-15.76%)
CY Booked + Current PnL: -14.91 L (-10.14%)
-------------------
Total profit:  1.92 L
Total loss:  -25.09 L
-------------------
Total Booked + Current PnL: 12.07 L (10.06%)
Total Booked PnL: 35.25 L (29.36%)
Curr Year Booked PnL: 8.27 L (6.28%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.14 C
Est FTT PnL: 82.65 L (62.76%)
Deployed:  1.20 C
Current:  1.32 C
CAGR/XIRR %: 5.96%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,26.66,H-SC,13.94,162319.0,36119.0,10064.0,-1.68,28.62,6.20,36.59,159.0,3.59,1.26,44.72,XY24,NTT,HEALTHCARE
68,SBIN,760.30,863.00,-10.56,M-LC,3.99,217114.0,15634.0,11572.0,1.86,7.76,5.33,13.51,62.0,1.35,1.68,21.33,XY25,NTT,BANKS
36,HINDALCO,651.95,756.01,-21.55,H-LC,5.73,107920.0,3608.0,13037.0,0.26,3.46,12.08,15.96,12.0,0.28,0.84,20.94,X5K,ATH,METALS
83,TTKPRESTIG,769.29,770.00,-8.91,M-SC,10.95,83316.0,-17461.0,17555.0,0.35,-17.33,21.07,0.09,228.0,-0.99,0.64,9.46,OX40N,NTT,DURABLES
27,DIXON,14367.00,18940.15,-30.14,H-MC,6.48,128632.0,13696.0,22884.0,1.36,11.92,17.79,31.83,85.0,0.60,1.00,37.90,X40N,ATH,IT


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,RAJOOENG,102.50,143.33,-31.84,H-SC,0.89,103240.0,740.0,40088.0,0.77,0.72,38.83,39.83,116.0,0.02,0.80,0.77,AR,ATH,MISC
41,ICICIPRULI,600.83,790.00,-15.66,H-MC,2.12,136222.0,434.0,42324.0,-1.20,0.32,31.07,31.48,110.0,0.01,1.05,12.48,X40,ATH,INSURANCE
25,DABUR,505.20,735.00,-9.52,H-MC,4.23,195780.0,-1248.0,90861.0,-1.03,-0.63,46.41,45.49,101.0,-0.01,1.52,12.32,XY24,BTT,FMCG
58,PGHH,13388.00,18062.58,-21.00,H-MC,6.55,199170.0,-1650.0,71761.0,0.42,-0.82,36.03,34.92,86.0,-0.02,1.54,3.82,X40,ATH,FMCG


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,CIPLA,1492.70,1795.0,-22.50,H-LC,1.94,203870.0,-630.0,42038.0,0.15,-0.31,20.62,20.25,10.0,-0.01,1.58,8.40,X40N,BTT,PHARMA
5,ASIANPAINT,2961.56,4250.0,-15.06,H-LC,2.41,168769.0,-32617.0,120231.0,0.25,-16.20,71.24,43.51,28.0,-0.27,1.31,17.48,X40,BTT,PAINTS
53,LTIM,5564.16,7230.2,-6.53,H-LC,3.61,180090.0,-20220.0,80194.0,-0.19,-10.09,44.53,29.94,24.0,-0.25,1.39,24.42,X200,ATH,IT
57,NESTLEIND,2268.60,2755.0,-59.66,H-LC,3.94,128033.0,-137393.0,194303.0,-0.20,-51.76,151.76,21.44,8.0,-0.71,0.99,3.55,XY25,NTT,FMCG
10,BAJAJHFL,122.26,152.0,-11.65,H-LC,3.99,183434.0,-17072.0,65853.0,-0.46,-8.51,35.90,24.33,29.0,-0.26,1.42,5.78,X40N,BTT,FINANCE


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TATAMOTORS,761.77,1065.00,-45.88,M-LC,5.54,235805.0,-40718.0,150797.0,2.51,-14.72,63.95,39.81,54.0,-0.27,1.83,13.01,XY24,NTT,AUTO
68,SBIN,760.30,863.00,-10.56,M-LC,3.99,217114.0,15634.0,11572.0,1.86,7.76,5.33,13.51,62.0,1.35,1.68,21.33,XY25,NTT,BANKS
85,UNIONBANK,123.87,163.00,-13.71,M-LC,15.11,151278.0,10438.0,34053.0,1.58,7.41,22.51,31.59,68.0,0.31,1.17,35.86,XY24,NTT,BANKS
27,DIXON,14367.00,18940.15,-30.14,H-MC,6.48,128632.0,13696.0,22884.0,1.36,11.92,17.79,31.83,85.0,0.60,1.00,37.90,X40N,ATH,IT
12,BANKINDIA,116.91,190.00,-34.97,H-MC,9.16,172410.0,-7398.0,119808.0,1.33,-4.11,69.49,62.52,90.0,-0.06,1.33,28.57,XR,NTT,BANKS


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,INDIGOPNTS,1407.73,1408.00,104.80,M-SC,3.71,134168.0,-40391.0,40425.0,-6.11,-23.14,30.13,0.02,224.0,-1.00,1.04,16.34,OX40N,NTT,PAINTS
90,VOLTAS,1278.28,1918.49,-31.38,H-MC,8.09,186840.0,-4902.0,100931.0,-4.54,-2.56,54.02,50.08,89.0,-0.05,1.45,4.07,XY25,ATH,AC
52,LAOPALA,369.40,464.00,105.71,H-SC,3.15,65343.0,-35503.0,61331.0,-3.25,-35.21,93.86,25.61,147.0,-0.58,0.51,21.15,AR,NTT,CERAMICS
29,EASEMYTRIP,18.12,26.40,-10.53,M-SC,22.95,86066.0,-90749.0,171547.0,-3.18,-51.32,199.32,45.70,199.0,-0.53,0.67,0.00,XY24,NTT,TRAVEL
84,UJJIVANSFB,52.77,60.00,34.19,H-SC,20.61,116208.0,-26271.0,45798.0,-2.40,-18.44,39.41,13.70,161.0,-0.57,0.90,37.16,OX40N,NTT,BANKS


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TTKPRESTIG,769.29,770.0,-8.91,M-SC,10.95,83316.0,-17461.0,17555.0,0.35,-17.33,21.07,0.09,228.0,-0.99,0.64,9.46,OX40N,NTT,DURABLES
84,UJJIVANSFB,52.77,60.0,34.19,H-SC,20.61,116208.0,-26271.0,45798.0,-2.40,-18.44,39.41,13.70,161.0,-0.57,0.90,37.16,OX40N,NTT,BANKS
20,CERA,7989.07,9475.0,-21.00,H-SC,3.04,112806.0,-30997.0,57745.0,0.99,-21.56,51.19,18.60,151.0,-0.54,0.87,24.48,OX40N,NTT,CERAMICS
72,SIS,477.00,528.0,2242.49,H-SC,7.06,63028.0,-18062.0,26730.0,-0.54,-22.27,42.41,10.69,157.0,-0.68,0.49,27.73,OX40N,NTT,MISC
49,KANSAINER,299.63,340.0,-70.03,H-SC,2.53,209286.0,-60381.0,96711.0,-0.15,-22.39,46.21,13.47,142.0,-0.62,1.62,5.85,XY24,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TCS,3794.03,4998.00,-27.38,H-LC,15.53,236340.0,-59594.0,153503.0,-0.21,-20.14,64.95,31.73,2.0,-0.39,1.83,0.90,X40,BTT,IT
89,VBL,492.64,671.64,-11.67,H-LC,10.96,267924.0,2884.0,93425.0,0.41,1.09,34.87,36.33,3.0,0.03,2.07,14.53,X40N,ATH,FMCG
45,INFY,1461.46,2275.00,-21.03,H-LC,9.86,253446.0,-8155.0,153791.0,-0.51,-3.12,60.68,55.67,4.0,-0.05,1.96,2.80,X40,BTT,IT
1,ABB,5551.76,7934.00,-45.31,H-LC,9.75,195312.0,-21207.0,114121.0,-0.28,-9.79,58.43,42.91,5.0,-0.19,1.51,2.29,AR,NTT,ELECTRICAL
57,NESTLEIND,2268.60,2755.00,-59.66,H-LC,3.94,128033.0,-137393.0,194303.0,-0.20,-51.76,151.76,21.44,8.0,-0.71,0.99,3.55,XY25,NTT,FMCG


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,26.66,H-SC,13.94,162319.0,36119.0,10064.0,-1.68,28.62,6.2,36.59,159.0,3.59,1.26,44.72,XY24,NTT,HEALTHCARE


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,26.66,H-SC,13.94,162319.0,36119.0,10064.0,-1.68,28.62,6.20,36.59,159.0,3.59,1.26,44.72,XY24,NTT,HEALTHCARE
50,KPIGREEN,497.21,731.35,25.90,H-SC,13.22,128948.0,3651.0,55357.0,1.08,2.91,42.93,47.09,148.0,0.07,1.00,61.39,MH,ATH,POWER
35,HEROMOTOCO,4311.81,5949.07,0.00,H-MC,6.04,160993.0,10080.0,47219.0,-1.31,6.68,29.33,37.97,96.0,0.21,1.25,33.67,AR,ATH,AUTO
15,BLUESTARCO,1646.70,2326.38,-5.73,H-SC,7.65,176860.0,12190.0,55782.0,-1.76,7.40,31.54,41.28,124.0,0.22,1.37,15.98,X40N,ATH,AC
19,CAMS,3643.00,5211.76,-7.47,H-SC,2.70,103914.0,1910.0,42012.0,-0.90,1.87,40.43,43.06,122.0,0.05,0.80,19.31,X40N,ATH,MISC


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,0.00,H-MC,6.04,160993.0,10080.0,47219.0,-1.31,6.68,29.33,37.97,96.0,0.21,1.25,33.67,AR,ATH,AUTO
42,INDIAMART,2327.09,4850.92,-48.45,H-SC,12.84,134376.0,11040.0,122726.0,-1.01,8.95,91.33,108.45,119.0,0.09,1.04,33.31,AR,ATH,MISC
50,KPIGREEN,497.21,731.35,25.90,H-SC,13.22,128948.0,3651.0,55357.0,1.08,2.91,42.93,47.09,148.0,0.07,1.00,61.39,MH,ATH,POWER
91,WHIRLPOOL,1219.98,2270.00,-43.86,M-SC,4.47,94545.0,3046.0,75702.0,-0.32,3.33,80.07,86.07,212.0,0.04,0.73,36.77,XR,NTT,DURABLES
61,RAJOOENG,102.50,143.33,-31.84,H-SC,0.89,103240.0,740.0,40088.0,0.77,0.72,38.83,39.83,116.0,0.02,0.80,0.77,AR,ATH,MISC


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-53.72,L-SC,22.20,68352.0,-45197.0,85249.0,-1.25,-39.80,124.72,35.27,267.0,-0.53,0.53,67.29,XR,NTT,HOTELS
3,ALKYLAMINE,2347.98,4546.37,-5.51,H-SC,17.39,87944.0,-13019.0,107547.0,-0.92,-12.89,122.29,93.63,150.0,-0.12,0.68,30.99,SR,ATH,CHEMICALS
54,MASFIN,326.60,399.50,-21.41,H-SC,15.82,89445.0,-8535.0,30402.0,-1.41,-8.71,33.99,22.32,152.0,-0.28,0.69,29.73,XR,ATH,FINANCE
36,HINDALCO,651.95,756.01,-21.55,H-LC,5.73,107920.0,3608.0,13037.0,0.26,3.46,12.08,15.96,12.0,0.28,0.84,20.94,X5K,ATH,METALS
84,UJJIVANSFB,52.77,60.00,34.19,H-SC,20.61,116208.0,-26271.0,45798.0,-2.40,-18.44,39.41,13.70,161.0,-0.57,0.90,37.16,OX40N,NTT,BANKS


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,SHALBY,261.39,327.00,931.90,M-SC,7.99,140436.0,-40707.0,86172.0,-0.31,-22.47,61.36,25.10,236.0,-0.47,1.09,12.55,XY24,NTT,HEALTHCARE
89,VBL,492.64,671.64,-11.67,H-LC,10.96,267924.0,2884.0,93425.0,0.41,1.09,34.87,36.33,3.0,0.03,2.07,14.53,X40N,ATH,FMCG
35,HEROMOTOCO,4311.81,5949.07,0.00,H-MC,6.04,160993.0,10080.0,47219.0,-1.31,6.68,29.33,37.97,96.0,0.21,1.25,33.67,AR,ATH,AUTO
51,LALPATHLAB,2838.53,3545.00,-23.40,H-SC,8.63,217790.0,24770.0,23260.0,0.39,12.83,10.68,24.89,137.0,1.06,1.69,39.18,X40N,NTT,HEALTHCARE
55,MEDANTA,1087.93,1486.00,26.66,H-SC,13.94,162319.0,36119.0,10064.0,-1.68,28.62,6.20,36.59,159.0,3.59,1.26,44.72,XY24,NTT,HEALTHCARE


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.45
1,25,41.08
2,50,70.85


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    41.31
LC    36.51
MC    22.17
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.05
X40N     15.30
X40      13.42
XY25     10.73
XR        8.96
AR        8.44
OX40N     7.58
X5K       2.24
MH        1.76
X200      1.39
SR        1.12
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    28.20
H-SC    26.39
H-MC    19.20
M-SC    13.48
M-LC     7.23
M-MC     2.62
L-SC     1.44
L-LC     1.08
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.28,-11.67,47.56
IT,11.99,-20.47,80.63
FINANCE,8.34,-20.94,65.68
BANKS,7.66,-15.13,64.01
MISC,5.94,-19.80,88.47
PAINTS,5.68,-19.15,42.64
HEALTHCARE,5.24,2.86,21.66
ELECTRICAL,4.70,-15.47,65.48
AUTO,4.46,-17.49,68.60


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2884813.0,23
XR,1133636.0,13
AR,1119040.0,10
X40,837394.0,10
XY25,663398.0,8
X40N,631196.0,12
OX40N,479668.0,10
SR,198696.0,2
X5K,149520.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2835924.0,29
H-LC,1510556.0,20
M-SC,1505881.0,17
H-MC,1249607.0,15
M-LC,444729.0,5
M-MC,338618.0,2
L-SC,274887.0,3
L-MC,57842.0,1
L-LC,47237.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      955899.0      7
M-SC       XY24      879458.0      7
H-SC       AR        672442.0      5
H-LC       X40       553642.0      6
H-SC       XR        533485.0      6
H-MC       XY24      480876.0      4
M-MC       XY24      338618.0      2
H-LC       X40N      324347.0      6
           XY25      257007.0      3
           AR        237217.0      2
H-MC       XY25      222680.0      2
H-SC       X40N      216445.0      4
H-MC       X40       207639.0      3
H-SC       OX40N     203600.0      4
           SR        198696.0      2
M-LC       XY24      184850.0      2
L-SC       XR        184146.0      2
M-SC       AR        162162.0      2
           OX40N     136715.0      4
M-LC       X5K       136483.0      1
M-SC       XR        126531.0      2
           XY25      124902.0      1
H-MC       XR        119808.0      1
M-LC       XR        111824.0      1
L-SC       OX40N      90741.0      1
H-MC       X40N       90404.0      2
H-LC       X200       80194.0      1
M-SC       X40        76113.0      1
L-MC       XR         57842.0      1
H-SC       MH         55357.0      1
H-MC       OX40N      48612.0      1
L-LC       XY25       47237.0      1
H-MC       AR         47219.0      1
H-LC       XY24       45112.0      1
H-MC       MH         32369.0      1
H-LC       X5K        13037.0      1
M-LC       XY25       11572.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 48.0 seconds
